# Snowpark For Python -- Advertising Spend and ROI Prediction

### In this session, we will cover:

* Creating Session object and securely connecting to Snowflake
* Loading data from Snowflake table into Snowpark DataFrame
* Perfoming Exploratory Data Analysis (EDA) on Snowpark DataFrame
* Pivoting and Merging datasets
* Creating Stored Procedure to deploy model training code on Snowflake
* Creating User-Defined Function (UDF) for inference

### Snowpark For Python Installation
- conda create --name snowpark -c https://repo.anaconda.com/pkgs/snowflake python=3.8
- conda activate snowpark
- pip install "snowflake-snowpark-python[pandas]"

### Other Libraries

- pip install ipykernel
- pip install scikit-learn

### Import Libraries

In [ ]:
# Snowpark for Python
from snowflake.snowpark.session import Session
from snowflake.snowpark.types import IntegerType, StringType, StructType, FloatType, StructField, DateType, Variant
from snowflake.snowpark.functions import udf, sum, col,array_construct,month,year,call_udf,lit
from snowflake.snowpark.version import VERSION
# Misc
import json
import pandas as pd

### Establish Secure Connection to Snowflake

 *Options: Username/Password, MFA, OAuth, Okta, SSO*

In [ ]:
# Create Snowflake Session object
connection_parameters = json.load(open('connection.json'))
session = Session.builder.configs(connection_parameters).create()

snowflake_environment = session.sql('select current_user(), current_role(), current_database(), current_schema(), current_version(), current_warehouse()').collect()
snowpark_version = VERSION

# Current Environment Details
print('User                        : {}'.format(snowflake_environment[0][0]))
print('Role                        : {}'.format(snowflake_environment[0][1]))
print('Database                    : {}'.format(snowflake_environment[0][2]))
print('Schema                      : {}'.format(snowflake_environment[0][3]))
print('Warehouse                   : {}'.format(snowflake_environment[0][5]))
print('Snowflake version           : {}'.format(snowflake_environment[0][4]))
print('Snowpark for Python version : {}.{}.{}'.format(snowpark_version[0],snowpark_version[1],snowpark_version[2]))

### Load and Examine Raw Click Data from Snowflake table

##### Notice the masked columns based on [Tag-based Masking Policies in Public Preview](https://docs.snowflake.com/en/user-guide/tag-based-masking-policies.html)

In [ ]:
snow_df_click_data = session.table('click_data').select(['COST','CLICK','IPADDRESS','MACADDRESS','TIMESTAMP'])
snow_df_click_data.show()

### Load Aggregated Campaign Spend Data from Snowflake table into Snowpark DataFrame

In [ ]:
snow_df_spend = session.table('campaign_spend')
snow_df_spend.show()

### Total Spend per Channel per Month

In [ ]:
# Stats per Month per Channel
snow_df_spend_per_channel = snow_df_spend.group_by(year('DATE'), month('DATE'),'CHANNEL').agg(sum('TOTAL_COST').as_('TOTAL_COST')).\
    with_column_renamed('"YEAR(DATE)"',"YEAR").with_column_renamed('"MONTH(DATE)"',"MONTH").sort('YEAR','MONTH')

snow_df_spend_per_channel.show(10)

### Pivot on Channel 

In [ ]:
snow_df_spend_per_month = snow_df_spend_per_channel.pivot('CHANNEL',['search_engine','social_media','video','email']).sum('TOTAL_COST').sort('YEAR','MONTH')
snow_df_spend_per_month = snow_df_spend_per_month.select(
    col("YEAR"),
    col("MONTH"),
    col("'search_engine'").as_("SEARCH_ENGINE"),
    col("'social_media'").as_("SOCIAL_MEDIA"),
    col("'video'").as_("VIDEO"),
    col("'email'").as_("EMAIL")
)
snow_df_spend_per_month.show()

### Merge Total Spend and Total Revenue per Month

In [ ]:
snow_df_revenue = session.table('monthly_revenue')
snow_df_revenue_per_month = snow_df_revenue.group_by('YEAR','MONTH').agg(sum('REVENUE')).sort('YEAR','MONTH').with_column_renamed('SUM(REVENUE)','REVENUE')
snow_df_spend_and_revenue_per_month = snow_df_spend_per_month.join(snow_df_revenue_per_month, ["YEAR","MONTH"])
snow_df_spend_and_revenue_per_month.show()

### >>>>>>>>>> *Examine Snowpark DataFrame Query and Explain Plan* <<<<<<<<<<

In [ ]:
snow_df_spend_and_revenue_per_month.explain()

### Features for Model Training
* Check for rows with missing values
* Exclude columns not needed for training a model
* Save features in Snowflake table

In [ ]:
# Delete rows with missing values if any
snow_df_spend_and_revenue_per_month = snow_df_spend_and_revenue_per_month.dropna()

# Exclude columns we don't need for modeling
snow_df_spend_and_revenue_per_month = snow_df_spend_and_revenue_per_month.drop(['YEAR','MONTH'])

# Save features in the Snowflake table
snow_df_spend_and_revenue_per_month.write.mode('overwrite').save_as_table('MARKETING_BUDGETS_FEATURES')
snow_df_spend_and_revenue_per_month.show()

### Model Training in Snowflake

#### Snowpark Python code to train model

In [ ]:
def train_revenue_prediction_model(session: Session, features_table: str) -> Variant:
    from sklearn.compose import ColumnTransformer
    from sklearn.pipeline import Pipeline
    from sklearn.preprocessing import PolynomialFeatures
    from sklearn.preprocessing import StandardScaler
    from sklearn.linear_model import LinearRegression
    from sklearn.model_selection import train_test_split, GridSearchCV

    import os
    from joblib import dump

    # Load features
    df = session.table(features_table).to_pandas()

    # Preprocess the Numeric columns
    # We apply PolynomialFeatures and StandardScaler preprocessing steps to the numeric columns
    # NOTE: High degrees can cause overfitting.
    numeric_features = ['SEARCH_ENGINE','SOCIAL_MEDIA','VIDEO','EMAIL']
    numeric_transformer = Pipeline(steps=[('poly',PolynomialFeatures(degree = 2)),('scaler', StandardScaler())])

    # Combine the preprocessed step together using the Column Transformer module
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, numeric_features)])

    # The next step is the integrate the features we just preprocessed with our Machine Learning algorithm to enable us to build a model
    pipeline = Pipeline(steps=[('preprocessor', preprocessor),('classifier', LinearRegression())])
    parameteres = {}

    X = df.drop('REVENUE', axis = 1)
    y = df['REVENUE']

    # Split dataset into training and test
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 42)

    # Use GridSearch to find the best fitting model based on 10 folds
    model = GridSearchCV(pipeline, param_grid=parameteres, cv=10)

    model.fit(X_train, y_train)

    # Upload trained model to a stage
    model_output_dir = '/tmp'
    model_file = os.path.join(model_output_dir, 'model.joblib')
    dump(model, model_file)
    session.file.put(model_file, "@dash_models",overwrite=True)

    # Return model R2 score on train and test data
    return {"R2 score on Train": model.score(X_train, y_train),"R2 score on Test": model.score(X_test, y_test)}

#### Test Python function

In [ ]:
train_revenue_prediction_model(session,"MARKETING_BUDGETS_FEATURES")

### Create Stored Procedure to deploy training code on Snowflake

In [ ]:
session.sproc.register(func=train_revenue_prediction_model,name="train_revenue_prediction_model",packages=['snowflake-snowpark-python','scikit-learn','joblib'],is_permanent=True,stage_location="@dash_sprocs",replace=True)

### Execute Stored Procedure to train model and deploy it on Snowflake

In [ ]:
print(session.call('train_revenue_prediction_model','MARKETING_BUDGETS_FEATURES'))

### >>>>>>>>>> *Examine Query History in Snowsight* <<<<<<<<<<

### Create User-Defined Function for inference

In [ ]:
session.clear_imports()
session.clear_packages()

# Add trained model as dependency
session.add_import('@dash_models/model.joblib.gz')

@udf(name='predict_roi',session=session,packages=['pandas','joblib','scikit-learn'],replace=True,is_permanent=True,stage_location='@dash_udfs')
def predict_roi(budget_allocations: list) -> float:
    import sys
    import pandas as pd
    from joblib import load

    IMPORT_DIRECTORY_NAME = "snowflake_import_directory"
    import_dir = sys._xoptions[IMPORT_DIRECTORY_NAME]
    
    model_file = import_dir + 'model.joblib.gz'

    model = load(model_file)
            
    features = ['SEARCH_ENGINE','SOCIAL_MEDIA','VIDEO','EMAIL']
    df = pd.DataFrame([budget_allocations], columns=features)

    roi = abs(model.predict(df)[0])

    return roi

### Call User-Defined Function for inference on sample data

In [ ]:
test_df = session.create_dataframe([[250000,250000,200000,450000],[500000,500000,500000,500000],[8500,9500,2000,500]], schema=['SEARCH_ENGINE','SOCIAL_MEDIA','VIDEO','EMAIL'])
test_df.select(
    'SEARCH_ENGINE','SOCIAL_MEDIA','VIDEO','EMAIL', 
    call_udf("predict_roi", array_construct(col("SEARCH_ENGINE"), col("SOCIAL_MEDIA"), col("VIDEO"), col("EMAIL"))).as_("PREDICTED_ROI")).show()

### >>>>>>>>>> *Examine Query History in Snowsight* <<<<<<<<<<

# Code on GitHub

### Python Notebook and Streamlit app is available at https://github.com/iamontheinet/dash-at-summit-2022